In [33]:
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random

from jax.scipy.linalg import expm
import numpy as np

import scipy

key = random.PRNGKey(0)
import qutip as qu


In [3]:
J = 0.3
c_ops = [J * qu.sigmax()]

H0 = 0.2*qu.sigmax()
psi0 = qu.basis(2, 0)
rho0 = qu.ket2dm(psi0)

projs = [np.array([[1, 0], [0, 0]]), np.array([[0, 0], [0, 1]])]


In [ ]:



def evolve_dm(rho0, H0, operators, t, no_qubits=1):
    liouvillian = qu.liouvillian(H0, c_ops=operators)
    rho_sup = qu.to_super(rho0)
    rho_evol = scipy.linalg.expm(liouvillian.full()*t)@rho_sup.full()
    dm_evol = rho_evol[:, 0].reshape(-1, 2**no_qubits)
    dm_evol = qu.Qobj(dm_evol)
    return dm_evol

In [ ]:

def partial_rho_gamma(rho0, H0, operators, t, bare_jump_op, no_qubits=1):
    liouvillian = qu.liouvillian(H0, c_ops=operators)
    dissipator = qu.lindblad_dissipator(bare_jump_op)

    print(dissipator)
    rho_sup = qu.to_super(rho0)
    rho_evol = (
        
            t*scipy.linalg.expm(liouvillian.full() * t) @ dissipator.full()  @rho_sup.full() 
       
        
    )
    # print(rho_evol)
    dm_evol = rho_evol[:, 0].reshape(-1, 2**no_qubits)
    print(dm_evol)
    dm_evol = qu.Qobj(dm_evol)
    return dm_evol


In [11]:
def make_evol_super(H0, ops):
    return (qu.liouvillian(H0, c_ops=ops) - qu.lindblad_dissipator(ops)).full()

In [17]:
B = qu.lindblad_dissipator(qu.sigmax()).full()

In [24]:
A = make_evol_super(H0, qu.sigmax())

C = qu.to_super(rho0).full()

In [76]:
gamma = 0.8

def evol(gamma, A, B, C):
    d = qu.basis(2, 0).full()
    return jnp.real(jnp.trace(projs[1]@(expm(A + gamma*B)@C)[:, 0].reshape(-1, 2)))

In [82]:
def evolve_dm(rho0, H0, operators, t, no_qubits=1):
    liouvillian = qu.liouvillian(H0, c_ops=operators)
    rho_sup = qu.to_super(rho0)
    rho_evol = expm(liouvillian.full()*t)@rho_sup.full()
    dm_evol = rho_evol[:, 0].reshape(-1, 2**no_qubits)
    # dm_evol = qu.Qobj(dm_evol)
    return dm_evol



In [83]:
grad(evolve_dm, 3)(rho0, H0, qu.sigmax(), 0.3)

TypeError: Gradient only defined for scalar-output functions. Output had shape: (2, 2).

In [77]:
grad(evol, 0)(9., A, B, C)

DeviceArray(8.940697e-08, dtype=float32, weak_type=True)

In [96]:
long_vector = jnp.array([jnp.arange(int(1e8)) for _ in range(2)])

%timeit jnp.dot(np.sum(long_vector), np.sum(long_vector)).block_until_ready()

9.08 ms ± 419 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [95]:
long_vector = np.array([np.arange(int(1e8)) for _ in range(2)])

%timeit np.dot(np.sum(long_vector), np.sum(long_vector))

165 ms ± 5.81 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [71]:
import jax
jax.devices()

[StreamExecutorGpuDevice(id=0, process_index=0)]

1

In [73]:
# ?jax.vmap

Signature:
jax.vmap(
    fun: 'F',
    in_axes: 'Union[int, Sequence[Any]]' = 0,
    out_axes: 'Any' = 0,
    axis_name: 'Optional[Hashable]' = None,
    axis_size: 'Optional[int]' = None,
    spmd_axis_name: 'Optional[Hashable]' = None,
) -> 'F'
Docstring:
Vectorizing map. Creates a function which maps ``fun`` over argument axes.

Args:
  fun: Function to be mapped over additional axes.
  in_axes: An integer, None, or (nested) standard Python container
    (tuple/list/dict) thereof specifying which input array axes to map over.

    If each positional argument to ``fun`` is an array, then ``in_axes`` can
    be an integer, a None, or a tuple of integers and Nones with length equal
    to the number of positional arguments to ``fun``. An integer or ``None``
    indicates which array axis to map over for all arguments (with ``None``
    indicating not to map any axis), and a tuple indicates which axis to map
    for each corresponding positional argument. Axis integers must be in the
  

In [110]:
from custom_models import simple_precession_with_noise

n = simple_precession_with_noise()

In [111]:
n.measurement_projectors

AttributeError: 'simple_precession_with_noise' object has no attribute 'measurement_projectors'